In [1]:
import numpy as np
import pandas as pd

In [ ]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [117]:
df=pd.read_csv('fake_job_postings.csv')

In [118]:
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [119]:
df['country']=df.location.str.split(',',expand=True)[:][0]
df['state']=df.location.str.split(',',expand=True)[:][1]
df['city']=df.location.str.split(',',expand=True)[:][2]

In [120]:
df['country'].sort_values().unique()

array(['AE', 'AL', 'AM', 'AR', 'AT', 'AU', 'BD', 'BE', 'BG', 'BH', 'BR',
       'BY', 'CA', 'CH', 'CL', 'CM', 'CN', 'CO', 'CY', 'CZ', 'DE', 'DK',
       'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GH', 'GR', 'HK', 'HR', 'HU',
       'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JP', 'KE', 'KH',
       'KR', 'KW', 'KZ', 'LK', 'LT', 'LU', 'LV', 'MA', 'MT', 'MU', 'MX',
       'MY', 'NG', 'NI', 'NL', 'NO', 'NZ', 'PA', 'PE', 'PH', 'PK', 'PL',
       'PT', 'QA', 'RO', 'RS', 'RU', 'SA', 'SD', 'SE', 'SG', 'SI', 'SK',
       'SV', 'TH', 'TN', 'TR', 'TT', 'TW', 'UA', 'UG', 'US', 'VI', 'VN',
       'ZA', 'ZM', nan], dtype=object)

In [121]:
missing=df[df.country.isna()==True]

 #19 866
len(missing[missing.fraudulent==1]), len(df[df.fraudulent==1])

(19, 866)

In [122]:
len(df['state'].sort_values().unique()),len(df['state'])

#----------processing
df.state = df.state.str.strip()
df.state.fillna(value='unknown', inplace=True)
df.state.replace(np.nan, 'unknown', inplace=True)
df.state.replace('', 'unknown', inplace=True)
df.state.replace(' ', 'unknown', inplace=True)
#---------end


df['state'].sort_values().unique()

array(['00', '01', '02', '03', '05', '06', '061', '07', '1', '10', '11',
       '110', '12', '13', '14', '16', '21', '22', '23', '25', '26', '28',
       '31', '34', '37', '40', '41', '44', '5', '51', '52', '56', '6',
       '61', '64', '65', '71', '8', '82', '84', '9', 'A', 'AA', 'AB',
       'ABD', 'ABE', 'ACT', 'AGB', 'AK', 'AL', 'ALA', 'ALX', 'AN', 'ANS',
       'ANT', 'AP', 'AR', 'ARM', 'AZ', 'B', 'BA', 'BAS', 'BC', 'BDF',
       'BE', 'BEN', 'BIR', 'BKM', 'BL', 'BMH', 'BNH', 'BPL', 'BR', 'BRC',
       'BRD', 'BRU', 'BRY', 'BS', 'BST', 'BU', 'BW', 'BY', 'C', 'CA',
       'CAM', 'CBF', 'CH', 'CHE', 'CHH', 'CHW', 'CJ', 'CMA', 'CMD', 'CN',
       'CO', 'COA', 'CON', 'CRF', 'CRY', 'CT', 'CTT', 'D', 'DA', 'DBY',
       'DC', 'DE', 'DER', 'DIF', 'DL', 'DNC', 'DND', 'DOR', 'DR', 'DS',
       'DU', 'DUD', 'DUR', 'E', 'EAW', 'EC', 'EDH', 'ENF', 'ENG', 'ER',
       'ERY', 'ES', 'ESS', 'ESX', 'F', 'FIF', 'FL', 'G', 'GA', 'GBN',
       'GE', 'GJ', 'GLG', 'GLS', 'GT', 'H', 'HAM', 'HCK', 'HE', 

In [123]:
print(df['city'].sort_values().unique().tolist())

[' ', '  ', '    Minneapolis', '   HG2 7NR', '  AB10', '  ALEXANDER CITY', '  Aberdeen ', '  Al Nashwa', '  COLUMBUS', '  Dayton', '  Edinburgh', '  Garner', '  Indianapolis', '  Kinnelon', '  Kreuzberg', '  Mclean VA.', '  Ottawa', '  Philadelphia ', '  Port Chester', '  Salt Lake City', '  San Diego', '  San Francisco', '  Vancouver', '  merseyside ', ' 02139', ' 04102', ' 10010', ' 10969', ' 15233', ' 2010', ' 20855', ' 28202', ' 32008', ' 32065', ' 33308', ' 33592', ' 33607', ' 380013', ' 43215', ' 46259  ACTON', ' 46802', ' 48226', ' 56073', ' 77619', ' 85032', ' 91401', ' 92101', ' 94306', ' 96080', ' :  Madison', ' : Columbus', ' A', ' ACADEMY LIFE', ' ACME MI  ', ' ACRA', ' AL Hasa', ' ALBANY', ' AMERSFOORT', ' ANNAPOLIS', ' ARDENVOIR WA  ', ' ARGYROUPOLIS', ' ARLINGTON', ' ATHENS', ' ATKINSON', ' ATLANTA', ' AUS/SAT', ' AUSTIN', ' Aarhus C', ' Aaronsburg ', ' Abbeville ', ' Abbeville  ', ' Abbot ', ' Abbotsford ', ' Abbott ', ' Abbottstown ', ' Abbyville ', ' Abell ', ' Abercr

In [101]:
len(df['city'].unique()),len(df['city']) #17880

#----
df.city = df.city.str.strip(' /:\\')
df.city.fillna(value='unknown', inplace=True)
df.city.replace(np.nan, 'unknown', inplace=True)
df.city.replace('', 'unknown', inplace=True)
df.city.replace(' ', 'unknown', inplace=True)
df['city']=df.city.str.lower()
#------

print(df['city'].sort_values().unique().tolist())

# 1. keep columns after processing
# 2. using the 3 un-processed columns

['02139', '04102', '10010', '10969', '15233', '2010', '20855', '28202', '32008', '32065', '33308', '33592', '33607', '380013', '43215', '46259  acton', '46802', '48226', '56073', '77619', '85032', '91401', '92101', '94306', '96080', 'a', 'aarhus c', 'aaronsburg', 'ab10', 'abbeville', 'abbot', 'abbotsford', 'abbott', 'abbottstown', 'abbyville', 'abell', 'abercrombie', 'aberdeen', 'aberdeenshire', 'abernant', 'abernathy', 'abie', 'abilene', 'absaraka', 'absarokee', 'absecon', 'abudhabi', 'academy life', 'acampo', 'accident', 'accokeek', 'accomac', 'accord', 'accra', 'acharnai', 'acharnes', 'ackerman', 'acme mi', 'acra', 'acworth', 'addison', 'adelaide', 'agia paraskevi', 'agios nikolaos', 'agios stefanos', 'ahens', 'ahmedabad', 'aix en provence', 'akron', 'al hasa', 'al hufuf', 'al nashwa', 'alabang', 'alabang area', 'alamo', 'alamosa', 'albany', 'albuquerque', 'albury', 'alexander city', 'alexandria', 'alfred', 'alimos', 'alk', 'all', 'all locations', 'all major cities', 'all of califor

In [124]:
#df['salary_range'].unique()


#df['min_salary']
#df['max_salary']
df['min_salary']=df.salary_range.str.split('-',expand=True)[:][0]
df['max_salary']=df.salary_range.str.split('-',expand=True)[:][1]



array([nan, '20000-28000', '100000-120000', '120000-150000',
       '50000-65000', '40000-50000', '60-80', '65000-70000', '75-115',
       '75000-110000', '17000-20000', '16000-28000', '95000-115000',
       '15000-18000', '50000-70000', '45000-60000', '30000-40000',
       '70000-90000', '10000-14000', '50-110', '28000-45000', '0-34300',
       '35000-40000', '9-Dec', '44000-57000', '18500-28000',
       '55000-75000', '30000-35000', '0-0', '20000-40000',
       '360000-600000', '50000-80000', '80000-100000', '52000-78000',
       '15750-15750', '40000-65000', '45000-50000', '30000-37000',
       '45000-67000', '35000-100000', '180000-216000', '45000-65000',
       '28000-32000', '0-1000', '36000-40000', '80000-110000',
       '35000-73000', '19000-19000', '60000-120000', '120000-15000000',
       '42000-55000', '90000-120000', '100000-150000', '28000-38000',
       '1600-1700', '50000-60000', '30000-70000', '32000-40000', '50-100',
       '9000-17000', '23040-28800', '105-110', '1300

In [113]:
df.min_salary.fillna(value='unknown', inplace=True)
df.min_salary.replace(np.nan, 'unknown', inplace=True)
df.min_salary.replace('', 'unknown', inplace=True)
df.min_salary.replace(' ', 'unknown', inplace=True)
df['min_salary'].sort_values().unique()

array(['0', '10', '100', '1000', '10000', '100000', '1000000', '104000',
       '105', '1050', '105000', '107000', '10716', '10800', '11', '110',
       '1100', '11000', '110000', '115', '1150', '115000', '1161', '120',
       '1200', '12000', '120000', '1234', '124000', '125', '12500',
       '125000', '13', '1300', '13000', '130000', '1300000', '135', '14',
       '140', '14000', '140000', '143520', '14400', '144000', '14436',
       '14500', '14560', '145600', '15', '150', '15000', '150000',
       '1500000', '1517', '15400', '15500', '15600', '156000', '15750',
       '16', '1600', '16000', '160000', '16500', '166400', '168000', '17',
       '17000', '175', '17500', '175000', '18', '18000', '180000',
       '18500', '18720', '19000', '190000', '1920', '19200', '192000',
       '19500', '196000', '19800', '2', '20', '200', '2000', '20000',
       '200000', '2000000', '20800', '208000', '21', '21000', '21120',
       '21600', '22', '22000', '220000', '23000', '23040', '24', '24000',


In [114]:
df.max_salary.fillna(value='unknown', inplace=True)
df.max_salary.replace(np.nan, 'unknown', inplace=True)
df.max_salary.replace('', 'unknown', inplace=True)
df.max_salary.replace(' ', 'unknown', inplace=True)
df['max_salary'].sort_values().unique()


# 1. use new column [median_salary]
# 2. use min & max


array(['0', '1', '100', '1000', '10000', '100000', '1000000', '102000',
       '105', '1050', '10500', '105000', '10800', '108000', '110', '1100',
       '11000', '110000', '110406', '114400', '115', '1150', '115000',
       '1161', '117000', '119000', '120', '1200', '12000', '120000',
       '1200000', '1200000000', '122720', '12345', '125', '12500',
       '125000', '126000', '13000', '130000', '13200', '135000',
       '1380000', '140', '1400', '14000', '140000', '1400000', '141000',
       '14400', '144000', '145000', '15', '150', '1500', '15000',
       '150000', '1500000', '15000000', '1517', '15600', '15750', '160',
       '1600', '16000', '160000', '16074', '165', '16500', '165000',
       '16500000', '168000', '17', '1700', '17000', '170000', '175',
       '17500', '175000', '17680', '18', '18000', '180000', '1800000',
       '18500', '18720', '187200', '19', '19000', '19200', '192000',
       '19600', '19760', '197600', '20', '200', '2000', '20000', '200000',
       '2000000'

In [ ]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'(?u)\b\w\w+\b')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords



In [ ]:
def vectorise_column(df, column):
    # Fill empty columns with "Unspecified"
    df[column] = df[column].fillna("Unspecified")
    
    # Create an instance of TfidfVectorizer
    vectoriser = TfidfVectorizer(analyzer=preprocess_text, ngram_range = (1,2))

    # Fit to the data and transform to feature matrix
    text_column = vectoriser.fit_transform(df[column])

    # Convert sparse matrix to dataframe
    text_column = pd.DataFrame.sparse.from_spmatrix(text_column)

    # Save mapping on which index refers to which words
    col_map = {v:k for k, v in vectoriser.vocabulary_.items()}

    # Rename each column using the mapping
    for col in text_column.columns:
        text_column.rename(columns={col: col_map[col]}, inplace=True)
    
    # Combined to dataframe
    combined = [df, text_column]
    df = pd.concat(combined, axis =1)
    
    return df

In [ ]:
# Create new table for text which can be merged to main table later
df_text = df.loc[:, ['company_profile', 'description', 'requirements', 'benefits']]
df_text['text'] = df_text.apply(lambda row: (str(row['company_profile']) + ' ' + str(row['description'])  
                                            + ' ' + str(row['requirements']) + ' ' + str(row['benefits'])), axis = 1)
to_merge = vectorise_column(df_text, 'text')

# Only the following columns need to be merged
# to_merge.iloc[:,4:]